In [1]:
# Install necessary packages
# Run the following commands to install the required libraries:
!pip install geopandas shapely

In [2]:
import geopandas as gpd
import requests


In [3]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for ODIAC dataset
collection_name = "odiac-ffco2-monthgrid-v2023"

In [4]:
# Read the state boundaries directly from the URL
url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
states_gdf = gpd.read_file(url)


In [5]:
# Transform CRS to WGS84 if not already
states_gdf = states_gdf.to_crs(epsg=4326)



In [6]:
# texas_aoi = {
#     "type": "Feature", # Create a feature object
#     "properties": {},
#     "geometry": { # Set the bounding coordinates for the polygon
#         "coordinates": [
#             [
#                 # [13.686159004559698, -21.700046934333145],
#                 # [13.686159004559698, -23.241974326585833],
#                 # [14.753560168039911, -23.241974326585833],
#                 # [14.753560168039911, -21.700046934333145],
#                 # [13.686159004559698, -21.700046934333145],
#                 [-95, 29], # South-east bounding coordinate
#                 [-95, 33], # North-east bounding coordinate
#                 [-104,33], # North-west bounding coordinate
#                 [-104,29], # South-west bounding coordinate
#                 [-95, 29]  # South-east bounding coordinate (closing the polygon)
#             ]
#         ],
#         "type": "Polygon",
#     },
# }

In [7]:
# Initialize a dictionary to hold the AOIs
state_aois = {}

# Loop through each state
for idx, row in states_gdf.iterrows():
    state_name = row['NAME']  # Get the state name
    geometry = row.geometry

    # Get the bounding box of the state's geometry
    minx, miny, maxx, maxy = geometry.bounds

    # Create the bounding box coordinates
    bbox_coords = [
        [minx, miny],  # Lower-left corner (min longitude, min latitude)
        [minx, maxy],  # Upper-left corner (min longitude, max latitude)
        [maxx, maxy],  # Upper-right corner (max longitude, max latitude)
        [maxx, miny],  # Lower-right corner (max longitude, min latitude)
        [minx, miny],  # Closing the polygon by returning to the first point
    ]

    # Construct the AOI dictionary
    # state_aoi = {
    #     "type": "Feature",
    #     "properties": {"name": state_name},
    #     "geometry": {
    #         "type": "Polygon",
    #         "coordinates": [bbox_coords],
    #     },
    # }

    state_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": { # Set the bounding coordinates for the polygon
        "coordinates": [
            bbox_coords
        ],
        "type": "Polygon",
    },
}

    # Add to the dictionary
    state_aois[state_name] = state_aoi


In [8]:
for state_name, aoi in state_aois.items():
    print(f"State: {state_name}")
    coords = aoi['geometry']['coordinates'][0]
    print("Bounding Coordinates:")
    for coord in coords:
        print(f"  {coord}")
    print()


State: Mississippi
Bounding Coordinates:
  [-91.65500899999999, 30.173942999999998]
  [-91.65500899999999, 34.996052]
  [-88.097888, 34.996052]
  [-88.097888, 30.173942999999998]
  [-91.65500899999999, 30.173942999999998]

State: North Carolina
Bounding Coordinates:
  [-84.32186899999999, 33.842316]
  [-84.32186899999999, 36.588117]
  [-75.460621, 36.588117]
  [-75.460621, 33.842316]
  [-84.32186899999999, 33.842316]

State: Oklahoma
Bounding Coordinates:
  [-103.002565, 33.615832999999995]
  [-103.002565, 37.002206]
  [-94.430662, 37.002206]
  [-94.430662, 33.615832999999995]
  [-103.002565, 33.615832999999995]

State: Virginia
Bounding Coordinates:
  [-83.675395, 36.540738]
  [-83.675395, 39.466012]
  [-75.242266, 39.466012]
  [-75.242266, 36.540738]
  [-83.675395, 36.540738]

State: West Virginia
Bounding Coordinates:
  [-82.644739, 37.201482999999996]
  [-82.644739, 40.638801]
  [-77.71951899999999, 40.638801]
  [-77.71951899999999, 37.201482999999996]
  [-82.644739, 37.20148299999

In [9]:
state_aois["Texas"]

{'type': 'Feature',
 'properties': {},
 'geometry': {'coordinates': [[[-106.645646, 25.837377],
    [-106.645646, 36.500704],
    [-93.508292, 36.500704],
    [-93.508292, 25.837377],
    [-106.645646, 25.837377]]],
  'type': 'Polygon'}}

In [10]:
texas_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": { # Set the bounding coordinates for the polygon
        "coordinates": [
            [
                # [13.686159004559698, -21.700046934333145],
                # [13.686159004559698, -23.241974326585833],
                # [14.753560168039911, -23.241974326585833],
                # [14.753560168039911, -21.700046934333145],
                # [13.686159004559698, -21.700046934333145],
                [-95, 29], # South-east bounding coordinate
                [-95, 33], # North-east bounding coordinate
                [-104,33], # North-west bounding coordinate
                [-104,29], # South-west bounding coordinate
                [-95, 29]  # South-east bounding coordinate (closing the polygon)
            ]
        ],
        "type": "Polygon",
    },
}
texas_aoi

{'type': 'Feature',
 'properties': {},
 'geometry': {'coordinates': [[[-95, 29],
    [-95, 33],
    [-104, 33],
    [-104, 29],
    [-95, 29]]],
  'type': 'Polygon'}}

In [11]:
# import geopandas as gpd
# import matplotlib.pyplot as plt
# from shapely.geometry import Polygon

# # Load Texas geometry
# url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
# states_gdf = gpd.read_file(url)
# states_gdf = states_gdf.to_crs(epsg=4326)
# texas_gdf = states_gdf[states_gdf['NAME'] == 'Texas']

# # Original approximate bounding box from initial script
# approx_bbox_coords = [
#     [-95, 29],
#     [-95, 33],
#     [-104, 33],
#     [-104, 29],
#     [-95, 29],
# ]
# approx_bbox_polygon = Polygon(approx_bbox_coords)

# # Generated actual bounding box from script
# minx, miny, maxx, maxy = texas_gdf.geometry.total_bounds
# actual_bbox_coords = [
#     [minx, miny],
#     [minx, maxy],
#     [maxx, maxy],
#     [maxx, miny],
#     [minx, miny],
# ]
# actual_bbox_polygon = Polygon(actual_bbox_coords)

# # Create GeoDataFrames for bounding boxes
# approx_bbox_gdf = gpd.GeoDataFrame({'geometry': [approx_bbox_polygon]}, crs='EPSG:4326')
# actual_bbox_gdf = gpd.GeoDataFrame({'geometry': [actual_bbox_polygon]}, crs='EPSG:4326')

# # Plotting
# fig, ax = plt.subplots(figsize=(10, 8))

# # Plot Texas
# texas_gdf.plot(ax=ax, color='lightgray', edgecolor='black')

# # Plot approximate bounding box
# approx_bbox_gdf.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=2, label='Approximate Bounding Box')

# # Plot actual bounding box
# actual_bbox_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Actual Bounding Box')

# plt.title('Comparison of Bounding Boxes for Texas')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.legend()
# plt.show()


In [12]:
# import geopandas as gpd

# # Load U.S. State Boundaries
# url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
# states_gdf = gpd.read_file(url)

# # Ensure CRS is WGS84
# states_gdf = states_gdf.to_crs(epsg=4326)

# # Initialize dictionary to hold AOIs
# state_aois = {}

# # Loop through each state to create bounding box AOIs
# for idx, row in states_gdf.iterrows():
#     state_name = row['NAME']
#     geometry = row.geometry

#     # Get bounding box
#     minx, miny, maxx, maxy = geometry.bounds

#     # Create bounding box coordinates
#     bbox_coords = [
#         [minx, miny],  # Lower-left
#         [minx, maxy],  # Upper-left
#         [maxx, maxy],  # Upper-right
#         [maxx, miny],  # Lower-right
#         [minx, miny],  # Close polygon
#     ]

#     # Create AOI dictionary
#     state_aoi = {
#         "type": "Feature",
#         "properties": {"name": state_name},
#         "geometry": {
#             "type": "Polygon",
#             "coordinates": [bbox_coords],
#         },
#     }

#     state_aois[state_name] = state_aoi


In [13]:
# Create a function that would search for a data collection in the US GHG Center STAC API

# First, we need to define the function
# The name of the function = "get_item_count"
# The argument that will be passed through the defined function = "collection_id"
def get_item_count(collection_id):

    # Set a counter for the number of items existing in the collection
    count = 0

    # Define the path to retrieve the granules (items) of the collection of interest in the STAC API
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    # Run a while loop to make HTTP requests until there are no more URLs associated with the collection in the STAC API
    while True:

        # Retrieve information about the granules by sending a "get" request to the STAC API using the defined collection path
        response = requests.get(items_url)

        # If the items do not exist, print an error message and quit the loop
        if not response.ok:
            print("error getting items")
            exit()

        # Return the results of the HTTP response as JSON
        stac = response.json()

        # Increase the "count" by the number of items (granules) returned in the response
        count += int(stac["context"].get("returned", 0))

        # Retrieve information about the next URL associated with the collection in the STAC API (if applicable)
        next = [link for link in stac["links"] if link["rel"] == "next"]

        # Exit the loop if there are no other URLs
        if not next:
            break

        # Ensure the information gathered by other STAC API links associated with the collection are added to the original path
        # "href" is the identifier for each of the tiles stored in the STAC API
        items_url = next[0]["href"]

    # Return the information about the total number of granules found associated with the collection
    return count

In [14]:
number_of_items = get_item_count(collection_name)

In [15]:
number_of_items

276

In [16]:
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]

In [17]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] is referring to the first item in the list/collection
items[0]

{'id': 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212/map?assets=co2-emissions&rescale=-10%2C60&colormap_name=jet',
   'rel': 'previ

In [18]:
# Now we create a dictionary where the start datetime values for each granule is queried more explicitly by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:7]: item for item in items}

# Next, we need to specify the asset name for this collection
# The asset name is referring to the raster band containing the pixel values for the parameter of interest
# For the case of the ODIAC Fossil Fuel CO₂ Emissions collection, the parameter of interest is “co2-emissions”
asset_name = "co2-emissions"

In [19]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [20]:
rescale_values

{'max': 31415.447265625, 'min': -675.1028442382812}

In [21]:
items['2020-01']['collection'], items['2020-01']['id']

('odiac-ffco2-monthgrid-v2023',
 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202001')

In [22]:
# # Texas, USA
# texas_aoi = {
#     "type": "Feature", # Create a feature object
#     "properties": {},
#     "geometry": { # Set the bounding coordinates for the polygon
#         "coordinates": [
#             [
#                 # [13.686159004559698, -21.700046934333145],
#                 # [13.686159004559698, -23.241974326585833],
#                 # [14.753560168039911, -23.241974326585833],
#                 # [14.753560168039911, -21.700046934333145],
#                 # [13.686159004559698, -21.700046934333145],
#                 [-95, 29], # South-east bounding coordinate
#                 [-95, 33], # North-east bounding coordinate
#                 [-104,33], # North-west bounding coordinate
#                 [-104,29], # South-west bounding coordinate
#                 [-95, 29]  # South-east bounding coordinate (closing the polygon)
#             ]
#         ],
#         "type": "Polygon",
#     },
# }

# '''
# State: Texas
# Bounding Coordinates:
#   [-106.645646, 25.837377]
#   [-106.645646, 36.500704]
#   [-93.508292, 36.500704]
#   [-93.508292, 25.837377]
#   [-106.645646, 25.837377]

# state_aoi = {
#         "type": "Feature",
#         "properties": {"name": state_name},
#         "geometry": {
#             "type": "Polygon",
#             "coordinates": [bbox_coords],
#         },
#     }

#     state_aois[state_name] = state_aoi


# ''';

In [23]:
# Check total number of items available within the collection
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=300"
).json()["features"]

# Print the total number of items (granules) found
print(f"Found {len(items)} items")

Found 276 items


In [24]:
# Examine the first item in the collection
items[0]

{'id': 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212/map?assets=co2-emissions&rescale=-10%2C60&colormap_name=jet',
   'rel': 'previ

In [25]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
# Create a function that retrieves information regarding a specific granule using its asset name and raster identifier and generates the statistics for it

# The function takes an item (granule) and a JSON (polygon) as input parameters
def generate_stats(item, geojson):

    # A POST request is made to submit the data associated with the item of interest (specific observation) within the boundaries of the polygon to compute its statistics
    result = requests.post(

        # Raster API Endpoint for computing statistics
        f"{RASTER_API_URL}/cog/statistics",

        # Pass the URL to the item, asset name, and raster identifier as parameters
        params={"url": item["assets"][asset_name]["href"]},

        # Send the GeoJSON object (polygon) along with the request
        json=geojson,

    # Return the response in JSON format
    ).json()

    # Return a dictionary containing the computed statistics along with the item's datetime information.
    return {
        **result["properties"],
        "start_datetime": item["properties"]["start_datetime"][:7],
    }

In [26]:
%%time
# %%time = Wall time (execution time) for running the code below

# Generate statistics using the created function "generate_stats" within the bounding box defined by the polygon
# This process may take a few minutes to complete
stats = [generate_stats(item, texas_aoi) for item in items]

CPU times: user 3.07 s, sys: 374 ms, total: 3.45 s
Wall time: 4min 32s


In [27]:
# Create a function that converts statistics in JSON format into a pandas DataFrame
import pandas as pd
def clean_stats(stats_json) -> pd.DataFrame:

    # Normalize the JSON data
    df = pd.json_normalize(stats_json)

    # Replace the naming "statistics.b1" in the columns
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]

    # Set the datetime format
    df["date"] = pd.to_datetime(df["start_datetime"])

    # Return the cleaned format
    return df

# Apply the generated function on the stats data
df = clean_stats(stats)

# Display the stats for the first 5 granules in the collection in the table
# Change the value in the parenthesis to show more or a smaller number of rows in the table
df.head(5)

,start_datetime,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_2,percentile_98,date
0,2022-12,0.743868,420840.18750,39.447887,177466.0,7000659.0,1908.116283,3.214699,64.481010,0.743868,160218.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.317880,220.576385,2022-12-01
1,2022-11,0.608937,343526.96875,32.262497,177466.0,5725496.5,1557.576964,2.635552,52.784710,0.608937,160204.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.081165,180.777283,2022-11-01
2,2022-10,0.617746,348376.68750,32.727840,177466.0,5808079.0,1579.566634,2.674915,53.548267,0.617746,160216.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.097266,183.435852,2022-10-01
3,2022-09,0.737875,417608.00000,39.143070,177466.0,6946564.0,1893.460654,3.190599,63.961494,0.737875,160221.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.308810,219.010132,2022-09-01
4,2022-08,0.910771,516733.53125,48.361969,177466.0,8582605.0,2342.897245,3.935843,78.948669,0.910771,160211.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.613806,270.067291,2022-08-01


In [28]:
state_stats = {}

for state_name, aoi in state_aois.items():
    print(f"Generating stats for {state_name}...")
    stats_list = []
    for item in items:
        try:
            stats = generate_stats(item, aoi)
            if stats is not None:
                stats_list.append(stats)
        except Exception as e:
            print(f"Error processing item {item.get('id', 'Unknown')} for {state_name}: {e}")
    state_stats[state_name] = stats_list

Generating stats for Mississippi...
Generating stats for North Carolina...
Generating stats for Oklahoma...
Generating stats for Virginia...
Generating stats for West Virginia...
Generating stats for Louisiana...
Generating stats for Michigan...
Generating stats for Massachusetts...
Generating stats for Idaho...
Generating stats for Florida...
Generating stats for Nebraska...
Generating stats for Washington...
Generating stats for New Mexico...
Generating stats for Puerto Rico...
Generating stats for South Dakota...
Generating stats for Texas...
Generating stats for California...
Generating stats for Alabama...
Generating stats for Georgia...
Generating stats for Pennsylvania...
Generating stats for Missouri...
Generating stats for Colorado...
Generating stats for Utah...
Generating stats for Tennessee...
Generating stats for Wyoming...


KeyboardInterrupt: 

In [29]:
state_stats['Mississippi']

[{'statistics': {'b1': {'min': 0.14518620073795319,
    'max': 318302.25,
    'mean': 15.079883575439453,
    'count': 156312.21875,
    'sum': 2357170.0,
    'std': 1051.7434929677484,
    'median': 2.3010239601135254,
    'majority': 1.3241674900054932,
    'minority': 0.14518620073795319,
    'unique': 101020.0,
    'histogram': [[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0],
     [0.14518620073795319,
      31830.353515625,
      63660.5625,
      95490.7734375,
      127320.984375,
      159151.1875,
      190981.390625,
      222811.609375,
      254641.8125,
      286472.03125,
      318302.25]],
    'valid_percent': 63.16,
    'masked_pixels': 91456.0,
    'valid_pixels': 156784.0,
    'percentile_2': 1.264420509338379,
    'percentile_98': 69.81523895263672}},
  'start_datetime': '2022-12'},
 {'statistics': {'b1': {'min': 0.1185138076543808,
    'max': 259826.484375,
    'mean': 12.326774597167969,
    'count': 156312.21875,
    'sum': 1926825.5,
    'std': 858.52733

In [30]:
all_stats = []
for state_name, stats_list in state_stats.items():
    for stats in stats_list:
        # Add the state name to the stats dictionary
        stats_with_state = stats.copy()  # Make a copy to avoid modifying the original
        stats_with_state['state'] = state_name
        # Append to the list
        all_stats.append(stats_with_state)



In [31]:
df = pd.DataFrame(all_stats)


In [32]:
df.head(5)

,statistics,start_datetime,state
0,"{'b1': {'min': 0.14518620073795319, 'max': 318...",2022-12,Mississippi
1,"{'b1': {'min': 0.1185138076543808, 'max': 2598...",2022-11,Mississippi
2,"{'b1': {'min': 0.12018692493438721, 'max': 263...",2022-10,Mississippi
3,"{'b1': {'min': 0.14407111704349518, 'max': 315...",2022-09,Mississippi
4,"{'b1': {'min': 0.17826856672763824, 'max': 390...",2022-08,Mississippi


In [33]:
# Normalize the 'statistics' column
stats_df = pd.json_normalize(df['statistics'])

# Rename columns if necessary (e.g., remove 'b1.' prefix)
stats_df.columns = [col.replace('b1.', '') for col in stats_df.columns]

# Concatenate the normalized statistics with the original DataFrame
df = pd.concat([df.drop(columns=['statistics']), stats_df], axis=1)


In [34]:
df.head(5)

,start_datetime,state,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_2,percentile_98
0,2022-12,Mississippi,0.145186,318302.250000,15.079884,156312.21875,2357170.000,1051.743493,2.301024,1.324167,0.145186,101020.0,"[[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...",63.16,91456.0,156784.0,1.264421,69.815239
1,2022-11,Mississippi,0.118514,259826.484375,12.326775,156312.21875,1926825.500,858.527336,1.883638,1.083975,0.118514,101013.0,"[[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...",63.16,91456.0,156784.0,1.035066,57.151360
2,2022-10,Mississippi,0.120187,263494.593750,12.502913,156312.21875,1954358.125,870.647747,1.910886,1.099655,0.120187,101015.0,"[[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...",63.16,91456.0,156784.0,1.050038,57.978081
3,2022-09,Mississippi,0.144071,315857.562500,14.961267,156312.21875,2338628.750,1043.665416,2.282485,1.313499,0.144071,101021.0,"[[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...",63.16,91456.0,156784.0,1.254233,69.252747
4,2022-08,Mississippi,0.178269,390831.062500,18.490070,156312.21875,2890224.000,1291.393482,2.817307,1.621272,0.178269,101016.0,"[[156775.0, 5.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...",63.16,91456.0,156784.0,1.548120,85.479744


In [35]:
df_selected = df[['state', 'start_datetime', 'max', 'mean']]



In [36]:
df_selected.head(5)

,state,start_datetime,max,mean
0,Mississippi,2022-12,318302.250000,15.079884
1,Mississippi,2022-11,259826.484375,12.326775
2,Mississippi,2022-10,263494.593750,12.502913
3,Mississippi,2022-09,315857.562500,14.961267
4,Mississippi,2022-08,390831.062500,18.490070


In [37]:
# Convert 'start_datetime' to datetime if not already
df_selected['start_datetime'] = pd.to_datetime(df_selected['start_datetime'])
# Extract the year
df_selected['year'] = df_selected['start_datetime'].dt.year
# Optionally drop 'start_datetime' if it's no longer needed
df_selected = df_selected.drop(columns=['start_datetime'])
# Reorder columns
df_selected = df_selected[['state', 'year', 'max', 'mean']]


<ipython-input-37-ee6a9bc6a974>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['start_datetime'] = pd.to_datetime(df_selected['start_datetime'])
<ipython-input-37-ee6a9bc6a974>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['year'] = df_selected['start_datetime'].dt.year


In [38]:
df_selected.head(5)

,state,year,max,mean
0,Mississippi,2022,318302.250000,15.079884
1,Mississippi,2022,259826.484375,12.326775
2,Mississippi,2022,263494.593750,12.502913
3,Mississippi,2022,315857.562500,14.961267
4,Mississippi,2022,390831.062500,18.490070


In [39]:
# Group by state and year, and calculate the mean of 'max' and 'mean'
yearly_stats = df_selected.groupby(['state', 'year'])[['max', 'mean']].mean().reset_index()


In [40]:
yearly_stats.head(5)

,state,year,max,mean
0,Alabama,2000,573175.906250,32.673104
1,Alabama,2001,564457.244792,32.179180
2,Alabama,2002,569756.218750,37.022653
3,Alabama,2003,575813.328125,37.415966
4,Alabama,2004,582954.489583,34.298451


In [41]:
yearly_stats['state'].unique()

array(['Alabama', 'California', 'Colorado', 'Florida', 'Georgia', 'Idaho',
       'Louisiana', 'Massachusetts', 'Michigan', 'Mississippi',
       'Missouri', 'Nebraska', 'New Mexico', 'North Carolina', 'Oklahoma',
       'Pennsylvania', 'Puerto Rico', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Virginia', 'Washington', 'West Virginia'],
      dtype=object)

In [42]:
yearly_stats['state'].nunique()

24

In [43]:
yearly_stats.to_csv('state_yearly_statistics.csv', index=False)